- $d\geq 2$ is the input data dimension
- $k$ is the number of cluster per dimensions (over $2$ dimensions)
- $n$ is the number of training samples
- $m$ is the number of neurons
- $p$ is cluster labels
- sd number of spurious dimensions

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import math
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from scripts.architecture import MLP, MLPManual
from scripts.train_utils import AverageMeter, accuracy
from scripts.plot_utils import plot_loss_accuracy, plotValAccuracy, fillSubplot
from scripts.optimizer import Optimizer
from scripts.train import *
from scripts.data import * 

In [ ]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
          'figure.figsize': (16, 8),
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(params)

matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18) 
matplotlib.rc('font', size=18)

In [ ]:
k = 3
d = 5
sd = d - 3
n = 1000
n_test = 4000

In [ ]:
num_epochs = 500
learning_rate = 0.5
loss_type = "Binary Cross Entropy"
loss_fn = torch.nn.BCELoss()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 1000
optim = "SGD"
measure_alignment = False
momentum, nesterov_momentum = False, False
weight_decay = None

In [ ]:
trainset, testset = syntheticData(k, n, n_test, sd)

In [ ]:
modelManual = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, True, False)
trainLostList_sgd1_scratch, trainAccList_sgd1_scratch, \
valLossList_sgd1_scratch, valAccList_sgd1_scratch,_,_  = train_model_manually(modelManual, k, trainset, testset, loss_type, loss_fn, num_epochs, batch_size, momentum,
                                                                         nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device, data="synthetic")

plot_loss_accuracy(trainLostList_sgd1_scratch,valLossList_sgd1_scratch,trainAccList_sgd1_scratch,valAccList_sgd1_scratch,num_epochs)

In [ ]:
modelManual = MLPManual(d, learning_rate, loss_type, "BP", None, optim, device, measure_alignment, False, False)

trainLostList_sgd1_scratch, trainAccList_sgd1_scratch, \
valLossList_sgd1_scratch, valAccList_sgd1_scratch,_,_  = train_model_manually(modelManual, k, trainset, testset, loss_type, loss_fn, num_epochs, batch_size, momentum,
                                                                              nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device, data="synthetic")

plot_loss_accuracy(trainLostList_sgd1_scratch,valLossList_sgd1_scratch,trainAccList_sgd1_scratch,valAccList_sgd1_scratch,num_epochs)

In [ ]:
modelManual = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", optim, device, measure_alignment, True, False)

trainLostList_sgd1_scratch, trainAccList_sgd1_scratch, \
valLossList_sgd1_scratch, valAccList_sgd1_scratch,_,_  = train_model_manually(modelManual, k, trainset, testset, loss_type, loss_fn, num_epochs, batch_size, momentum,
                                                                              nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device, data="synthetic")

plot_loss_accuracy(trainLostList_sgd1_scratch,valLossList_sgd1_scratch,trainAccList_sgd1_scratch,valAccList_sgd1_scratch,num_epochs)

In [ ]:
num_epochs = 500
ns = [32,64,128,256,512]
n_test = 4000

In [ ]:
# please note that changing the structure of data (it changes for each run)
# will give you different results, to reproduce the experiments we've set layer 
# size 1000
trainset, testset = syntheticData(k, n_max, n_test, sd)

In [ ]:
d = 15 # similar to paper
sd = d - 3
idx = 0
n_max = max(ns)
df = pd.DataFrame(columns=["Test Accuracy", "n", "Method"])

for i in range(1,4): 
    for n in ns:
        print("Number of data points:", n)
        modelManual1 = MLPManual(d, learning_rate, loss_type, "BP", None, "SGD", device, measure_alignment, True, False)
        modelManual2 = MLPManual(d, learning_rate, loss_type, "BP", None, "SGD", device, measure_alignment, False, False)
        # modelManual1 = MLPManual(d, learning_rate, loss_type, "BP", None, "Adam", device, measure_alignment, True, False)
        # modelManual2 = MLPManual(d, learning_rate, loss_type, "BP", None, "RMSProp", device, measure_alignment, True, False)
        # modelManual3 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", "Adam", device, measure_alignment, True, False)
        # modelManual4 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", "RMSProp", device, measure_alignment, True, False)
        results = {}
        for model in [modelManual1, modelManual2]:    
            trainLostList_sgd1_scratch1, trainAccList_sgd1_scratch1, \
            valLossList_sgd1_scratch1, valAccList_sgd1_scratch1,_,_  = train_model_manually(model, k, trainset, testset, loss_type, loss_fn, num_epochs, n, momentum,
                                                                                            nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device,
                                                                                            data="synthetic")
            results[i] = valAccList_sgd1_scratch1

            liste = []
            for i in results:
                liste.append(results[i][-1])

            method = model.train_method + "_" + model.optim + "_" + str(model.update_both)

            for value in liste:
                df.loc[idx,:] = [value, n, method]
                idx += 1

df["Error"] = df["Test Accuracy"].apply(lambda x: 1-x)
df.to_csv("runs/syntheticData_n.csv", index=False)

In [ ]:
n = 256
ds = [5,10,15,20,25]
num_epochs = 500

In [ ]:
idx = 0
df = pd.DataFrame(columns=["Test Accuracy", "d", "Method"])
sd = max(ds) - 3

for i in range(1,4):  
    for d in ds:
        print("Number of dimensions points:", d)
        modelManual1 = MLPManual(d, learning_rate, loss_type, "BP", None, "SGD", device, measure_alignment, True, False)
        modelManual2 = MLPManual(d, learning_rate, loss_type, "BP", None, "SGD", device, measure_alignment, False, False)
        # modelManual3 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", "SGD", device, measure_alignment, True, False)
        # modelManual4 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", "RMSProp", device, measure_alignment, True, False)
        # modelManual5 = MLPManual(d, learning_rate, loss_type, "BP", None, "RMSProp", device, measure_alignment, True, False)
            
        results = {} 
        for model in [modelManual1, modelManual2]:
            trainLostList_sgd1_scratch1, trainAccList_sgd1_scratch1, \
            valLossList_sgd1_scratch1, valAccList_sgd1_scratch1,_,_  = train_model_manually(model, k, trainset, testset, loss_type, loss_fn, num_epochs, n, momentum,
                                                                                            nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device,
                                                                                            data="synthetic")
            results[i] = valAccList_sgd1_scratch1

            liste = []
            for i in results:
                liste.append(results[i][-1])

            method = model.train_method + "_" + model.optim + "_" + str(model.update_both)

            for value in liste:
                df.loc[idx,:] = [value, d, method]
                idx += 1

df["Error"] = df["Test Accuracy"].apply(lambda x: 1-x)
df.to_csv("runs/syntheticData_d.csv", index=False)

In [ ]:
# reproduced experiment from paper
X1 = trainset[:][0][trainset[:][1] == 1, :]
X2 = trainset[:][0][trainset[:][1] == 0, :]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,8))
ax1.plot(X1[:,1],X1[:,2],"+r")
ax1.plot(X2[:,1],X2[:,2],"ob", markerfacecolor='none')
ax1.plot(cluster_center(torch.arange(0,k**2+1),k)[0],cluster_center(torch.arange(0,k**2+1),k)[1],"ok")
ax1.set_title("Distribution")
ax1.axis("equal")
ax1.axis("off")


df_n = pd.read_csv("runs/syntheticData_n.csv")
df_n = df_n[df_n["Method"].apply(lambda x: "DFA" not in x)]
g = sns.pointplot(data=df_n, x="n", y="Error", hue="Method", alpha=.6, ax=ax2)
g.legend_.set_title(None)
ax2.legend()
ax2.set_ylabel("Test \nError", rotation=0, labelpad=30)
ax2.yaxis.set_label_coords(-0.2,0.5)
ax2.set_xlabel("n", rotation=0, labelpad=30)
ax2.grid()
ax2.set_title("Test Error vs. $n$")
ax2.set_ylim(0,0.5)
ax2.set_yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5])


df_d = pd.read_csv("runs/syntheticData_d.csv")
df_d = df_d[df_d["Method"].apply(lambda x: "DFA" not in x)]
g = sns.pointplot(data=df_d, x="d", y="Error", hue="Method", alpha=.6, ax=ax3)
ax3.get_legend().remove()
ax3.set_ylabel("")
ax3.set_xlabel("d", rotation=0, labelpad=30)
ax3.grid()
ax3.set_title("Test error vs. $d$")
ax3.set_ylim(0,0.5)
ax3.set_yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5]);

fig.tight_layout()
fig.savefig("plots/syntheticData_reproduced.png")
#fig.savefig("../../2-writing/oxforddown/figures/2_syntheticData_reproduced.png")

In [ ]:
# set layer_size = 512
idx = 0
df = pd.DataFrame(columns=["Test Accuracy", "d", "Method"])
sd = max(ds) - 3

for i in range(1,4):  
    for d in ds:
        print("Number of dimensions points:", d)
        modelManual1 = MLPManual(d, learning_rate, loss_type, "BP", None, "SGD", device, measure_alignment, True, False)
        modelManual3 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", "SGD", device, measure_alignment, True, False)
        modelManual4 = MLPManual(d, learning_rate, loss_type, "DFA", "uniform", "RMSProp", device, measure_alignment, True, False)
        modelManual5 = MLPManual(d, learning_rate, loss_type, "BP", None, "RMSProp", device, measure_alignment, True, False)
            
        results = {} 
        for model in [modelManual1, modelManual3, modelManual4, modelManual5]:
            trainLostList_sgd1_scratch1, trainAccList_sgd1_scratch1, \
            valLossList_sgd1_scratch1, valAccList_sgd1_scratch1,_,_  = train_model_manually(model, k, trainset, testset, loss_type, loss_fn, num_epochs, n, momentum,
                                                                                            nesterov_momentum, weight_decay, measure_alignment, n,d, validate_model = True, device=device,
                                                                                            data="synthetic")

            results[i] = valAccList_sgd1_scratch1

            liste = []
            for i in results:
                liste.append(results[i][-1])

            method = model.train_method + "_" + model.optim + "_" + str(model.update_both)

            for value in liste:
                df.loc[idx,:] = [value, d, method]
                idx += 1

df["Error"] = df["Test Accuracy"].apply(lambda x: 1-x)
df.to_csv("runs/syntheticData_d_adap.csv", index=False)

In [ ]:
n_max = 512
sd = 22
# trainset, testset = syntheticData(k, n_max, n_test, sd)

In [ ]:
fig, (ax2, ax3) = plt.subplots(1, 2, figsize=(16,8))

X1 = trainset[:][0][trainset[:][1] == 1, :]
X2 = trainset[:][0][trainset[:][1] == 0, :]

ax2.plot(X1[:,1],X1[:,2],"+r")
ax2.plot(X2[:,1],X2[:,2],"ob", markerfacecolor='none')
ax2.plot(cluster_center(torch.arange(0,k**2+1),k)[0],cluster_center(torch.arange(0,k**2+1),k)[1],"ok")
ax2.set_title("Distribution")
ax2.axis("equal")
ax2.axis("off")

df_n = pd.read_csv("runs/syntheticData_d_adap.csv")
g = sns.pointplot(data=df_n, x="d", y="Error", hue="Method", alpha=.6, ax=ax3)
g.legend_.set_title(None)
ax3.legend(loc="lower right")
ax3.set_ylabel("Test \nError", rotation=0, labelpad=30)
ax3.yaxis.set_label_coords(-0.15,0.5)
ax3.set_xlabel("d", rotation=0, labelpad=30)
ax3.grid()
ax3.set_title("Test Error vs. $d$")
ax3.set_ylim(0,0.5)
ax3.set_yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
fig.savefig("plots/syntheticData_adaptive.png")
#fig.savefig("../../2-writing/oxforddown/figures/3_syntheticData_adaptive.png");